In [1]:
import numpy as np
import logging
import sys
from uniswapv3_simulator.pool import Uniswapv3Pool
from uniswapv3_simulator.math import tick_to_sqrt_price

logging.basicConfig(level=logging.INFO, 
                    stream=sys.stdout)  # ensures logging and print statements are in correct order
logging.getLogger('uniswap-v3').setLevel(logging.DEBUG)

**Notes:**
1. `token0` = $x$, `token1` = $y$. $P = \frac{y}{x}$.
2. Swapping in `token1` (receiving `token0`) results in the price increasing as $y$ increases and $x$ decreases (i.e., move to the right).
2. Swapping in `token0` (receiving `token1`) results in the price decreasing as $y$ decreases and $x$ increases (i.e., move to the left).

**Test 1:** LP with a single position. Two swaps are executed, both within the the range of liquidity provided by the LP. LP removes all liquidity (and is able to collect fees).

In [2]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 0.1 of token0
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 0.1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 1 of token1
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 1)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove all liquidity from the pool
# token0, token1 output from set_position are positive, indicating the amount
# the LP received from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}')

# total fees owed to LP due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# fees owed to the single LP should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 2:** LP with a single position. Two swaps are executed. The first within the range of liquidity provided by the LP. The second exhausts all liquidity of token0. LP removes all liquidity (and is able to collect fees).

In [3]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 0.1 of token0
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 0.1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 5 of token1
# this swap should exhaust all of the liquidity in the pool, leaving
# the pool with 0 of token0 and x of token1
# the swap will also only be partially executed as there is not enough 
# of token0 to execute the desired trade
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 5)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove all liquidity from the pool
# token0, token1 output from set_position are positive, indicating the amount
# the LP received from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}')

# total fees owed to LP due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# fees owed to the single LP should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 3:** LP with a single position. Two swaps are executed. The first within the range of liquidity provided by the LP. The second exhausts all liquidity of token1. LP removes all liquidity (and is able to collect fees).

In [4]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 1 of token1
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 1)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 1 of token0
# this swap should exhaust all of the liquidity in the pool, leaving
# the pool with x of token0 and 0 of token1
# the swap will also only be partially executed as there is not enough 
# of token0 to execute the desired trade
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove all liquidity from the pool
# token0, token1 output from set_position are positive, indicating the amount
# the LP received from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}')

# total fees owed to LP due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# fees owed to the single LP should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 4:** LP with a single position. Swaps alternate between exhausting each token, and try to execute swaps when the pool is missing required tokens. LP removes all liquidity (and is able to collect fees). 

In [5]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 5 of token1
# this swap should exhaust all of the liquidity in the pool, leaving
# the pool with 0 of token0 and x of token1
# the swap will also only be partially executed as there is not enough 
# of token0 to execute the desired trade
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 5)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# swap 1 of token1
# this swap should not execute as we are already at a limit point
token0, token1 = pool.swap(1, 1)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 1 of token0
# this swap should exhaust all of the liquidity in the pool, leaving
# the pool with x of token0 and 0 of token1
# the swap will also only be partially executed as there is not enough 
# of token0 to execute the desired trade
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# swap 0.1 of token0
# this swap should not execute as we are already at a limit point
token0, token1 = pool.swap(0, 0.1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove all liquidity from the pool
# token0, token1 output from set_position are positive, indicating the amount
# the LP received from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}')

# total fees owed to LP due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# fees owed to the single LP should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 5:** Two LP positions. Two swaps are executed, both within the the range of liquidity provided by the LP. LP removes all liquidity (and is able to collect fees).

In [6]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# add a second position with a larger range
# both positions have the same liquidity, so this position will need to be 
# contribute more tokens
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick + 4, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 0.1 of token0
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 0.1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 1 of token1
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 1)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove the first position from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# remove the second position from the pool
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick + 4, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}\n')

# total fees owed to first position due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP - position 1')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# total fees owed to second position due to transaction(s)
position = pool.position_map[('id1', start_tick - 4, start_tick + 4)]
print('Fees owed to LP - position 2')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}\n')

# since both positions have the same liquidity and transactions
# occured within both positions' ranges, the two positions
# should have the same fees
# total fees owed to the two LP positions should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 6:** Two LP positions, but one is initialized after the first swap. Two swaps are executed, both within the the range of liquidity provided by the LP. LP removes all liquidity (and is able to collect fees).

In [7]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 0.1 of token0
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 0.1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# add a second position with a larger range
# both positions have the same liquidity, so this position will need to be 
# contribute more tokens
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick + 4, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# swap 1 of token1
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 1)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# liquidity will be different now as we have added a position
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after position 2 added and swap 2 executed')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove the first position from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# remove the second position from the pool
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick + 4, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}\n')

# total fees owed to first position due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP - position 1')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# total fees owed to second position due to transaction(s)
position = pool.position_map[('id1', start_tick - 4, start_tick + 4)]
print('Fees owed to LP - position 2')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}\n')

# the first position should have the fees from the first swap while the 
# fees from the second swap should be split evenly between the positions
# total fees owed to the two LP positions should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 7:** Two LP positions. Two swaps are executed, the first is within the the range of liquidity of both positions, but the second crosses the tick of the first position. LP removes all liquidity (and is able to collect fees).

In [8]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}\n')

# contribute liquidity to the pool
# token0, token1 output from set_position are negative, indicating the amount
# the LP must provide tokens to the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# add a second position with a larger range
# both positions have the same liquidity, so this position will need to be 
# contribute more tokens
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick + 4, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the initial liquidity values
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 0.1 of token0
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 0.1)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 10 of token1
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 10)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# now that we crossed the first tick, liquidity should be reduced 
# from 20,000 -> 10,000
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove the first position from the pool
token0, token1 = pool.set_position('id1', start_tick - 2, start_tick + 2, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# remove the second position from the pool
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick + 4, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}\n')

# total fees owed to first position due to transaction(s)
position = pool.position_map[('id1', start_tick - 2, start_tick + 2)]
print('Fees owed to LP - position 1')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}')

# total fees owed to second position due to transaction(s)
position = pool.position_map[('id1', start_tick - 4, start_tick + 4)]
print('Fees owed to LP - position 2')
print(f'Fees owed in token0: {position.tokens_owed0:,.4f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.4f}\n')

# fees from first swap (token0) should be split evenly between positions
# as swap occured within the the range of both positions
# the second position should have a larger share of the fees from the
# second swap (token1) as part of the swap was executed within the shared
# range, but the upper tick for the first position was crossed and the 
# rest of the swap occured where only the second position provided liquidity
# so the remaining fees went to the second position
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027

DEBUG:uniswap-v3.tick:Tick 23,025 initialized (sqrt_price=3.1620).
DEBUG:uniswap-v3.tick:Tick 23,029 initialized (sqrt_price=3.1626).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,025, 23,029 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,025.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,029.
DEBUG:uniswap-v3.pool:liquidity_delta added to liquidity.
DEBUG:uniswap-v3.pool:Updated liquidity: 10,000.00.
DEBUG:uniswap-v3.pool:token0 added to/remove

**Test 8:** Two LP positions side by side (no shared range). Two swaps are executed, crossing between the position ranges. LP removes all liquidity (and is able to collect fees).

In [9]:
# initialize pool
pool = Uniswapv3Pool(0.01, 1, 10)
start_tick = pool.tick
print(pool)
print(f'Starting tick: {start_tick:,}')
print(f'Starting tick sqrt_price: {tick_to_sqrt_price(start_tick):,.8f}')
print(f'Start sqrt_price:         {pool.sqrt_price:,.8f}\n')

# contribute liquidity to the pool on one side of the current price/tick
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# add a second position to the on the other side of the current price/tick
token0, token1 = pool.set_position('id1', start_tick, start_tick + 4, 10000)
print(f'LP must provide {-token0:,.4f} of token0 and {-token1:,.4f} of token1\n')

# check the total tokens in the pool
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}\n')

# check the initial liquidity values
# the start price is a little above tick 23,027, so the price is within the
# second position's range so we should only have 10,000 of liquidity
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves before any swaps')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 0.2 of token0
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token0 is negative as the trader provides this to the pool
# token1 is positive as the trader receives this from the pool
token0, token1 = pool.swap(0, 0.2)
print(f'Trader swapped {-token0:,.4f} of token0 for {token1:,.4f} of token1\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 1')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# swap 5 of token1
# this swap should execute entirely within the current tick range 
# i.e., no ticks should need to be crossed
# token1 is negative as the trader provides this to the pool
# token0 is positive as the trader receives this from the pool
token0, token1 = pool.swap(1, 5)
print(f'Trader swapped {-token1:,.4f} of token1 for {token0:,.4f} of token0\n')

# since we have not crossed any ticks, liquidity should not have changed
x, y = pool.virtual_reserves
print('Liquidity and virtual reserves after swap 2')
print(f'x: {x:,.2f}')
print(f'y: {y:,.2f}')
print(f'Calculated pool liquidity: {np.sqrt(x * y):,.2f}')
print(f'Actual pool liquidity: {pool.liquidity:,.2f}\n')

# remove the first position from the pool
token0, token1 = pool.set_position('id1', start_tick - 4, start_tick, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# remove the second position from the pool
token0, token1 = pool.set_position('id1', start_tick, start_tick + 4, -10000)
print(f'LP received {token0:,.4f} of token0 and {token1:,.4f} of token1\n')

# since all liquidity was removed, the pool should have 0 
# of token0 and token1
print('Total tokens in pool')
print(f'Pool token0: {pool.token0:,.4f}')
print(f'Pool token1: {pool.token1:,.4f}\n')

# total fees owed to first position due to transaction(s)
position = pool.position_map[('id1', start_tick - 4, start_tick)]
print('Fees owed to LP - position 1')
print(f'Fees owed in token0: {position.tokens_owed0:,.8f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.8f}')

# total fees owed to second position due to transaction(s)
position = pool.position_map[('id1', start_tick, start_tick + 4)]
print('Fees owed to LP - position 2')
print(f'Fees owed in token0: {position.tokens_owed0:,.8f}')
print(f'Fees owed in token1: {position.tokens_owed1:,.8f}\n')

# since both positions have the same liquidity and transactions
# occured within both positions' ranges, the two positions
# should have the same fees
# total fees owed to the two LP positions should match total fees collected
# by the pool as well
print('Fees collected by pool')
print(f'Fees collected in token0: {pool.total_fees_token0:,.4f}')
print(f'Fees collected in token1: {pool.total_fees_token1:,.4f}')

Pool(price=10.0000, liquidity=0.00, fee=1.00%)
Starting tick: 23,027
Starting tick sqrt_price: 3.16227731
Start sqrt_price:         3.16227766

DEBUG:uniswap-v3.tick:Tick 23,023 initialized (sqrt_price=3.1616).
DEBUG:uniswap-v3.tick:Tick 23,027 initialized (sqrt_price=3.1623).
DEBUG:uniswap-v3.pool:token0 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.pool:token1 fee growth inside position range: 0.00000000.
DEBUG:uniswap-v3.position:Position id1, 23,023, 23,027 initialized.
DEBUG:uniswap-v3.position:token0 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:token1 uncollected fees: 0.0000.
DEBUG:uniswap-v3.position:Updated position liquidity: 10,000.00.
DEBUG:uniswap-v3.position:Total token0 owed: 0.0000.
DEBUG:uniswap-v3.position:Total token1 owed: 0.0000.
DEBUG:uniswap-v3.tick:10,000.00 liquidity added to tick 23,023.
DEBUG:uniswap-v3.tick:10,000.00 liquidity subtracted from tick 23,027.
DEBUG:uniswap-v3.pool:token0 added to/removed from pool: 0.0000
DEBUG:uniswap-v3

DEBUG:uniswap-v3.pool:token1 added to/removed from pool: -2.9736
DEBUG:uniswap-v3.pool:Tick 23,027 no longer has liquidity referencing it and was deleted.
DEBUG:uniswap-v3.pool:Tick 23,031 no longer has liquidity referencing it and was deleted.
LP received 0.3350 of token0 and 2.9736 of token1

Total tokens in pool
Pool token0: 0.0000
Pool token1: 0.0000

Fees owed to LP - position 1
Fees owed in token0: 0.00199648
Fees owed in token1: 0.01996356
Fees owed to LP - position 2
Fees owed in token0: 0.00000352
Fees owed in token1: 0.03003644

Fees collected by pool
Fees collected in token0: 0.0020
Fees collected in token1: 0.0500
